In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import pandas as pd
import numpy as np
# from prophet import Prophet
# from prophet.diagnostics import cross_validation, performance_metrics
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import holidays
import os
import pickle
import torch
from torch.utils.data import Dataset, DataLoader


In [ ]:
import pandas as pd

# Load the first DataFrame
df1 = pd.read_pickle('df_4_files_combined_no_outliers_for_AR.pkl')
df1.index = pd.to_datetime(df1.index)
df1 = df1.asfreq('D')

# Load the second DataFrame
df2 = pd.read_pickle('df_file_combined_actual_no_outliers_for_AR.PKL')
df2.index = pd.to_datetime(df2.index)
df2 = df2.asfreq('D')

# Remove overlap (keep only rows in df2 that are not in df1)
df2_unique = df2[~df2.index.isin(df1.index)]

# Merge the DataFrames
merged_df = pd.concat([df1, df2_unique]).sort_index()

# Optional: save merged DataFrame
merged_df.to_pickle('merged_df_no_overlap.pkl')

print("Merged DataFrame shape:", merged_df.shape)


In [ ]:
import pandas as pd

# Load the first DataFrame
df1 = pd.read_pickle('df_4_files_combined_no_outliers_for_AR.pkl')
df1.index = pd.to_datetime(df1.index)
df1 = df1.asfreq('D')

# Load the second DataFrame
df2 = pd.read_pickle('df_file_combined_actual_no_outliers_for_AR.PKL')
df2.index = pd.to_datetime(df2.index)
df2 = df2.asfreq('D')

# Remove overlap (keep only rows in df2 that are not in df1)
df2_unique = df2[~df2.index.isin(df1.index)]

# Merge the DataFrames
merged_df = pd.concat([df1, df2_unique]).sort_index()

# Optional: save merged DataFrame
merged_df.to_pickle('merged_df_no_overlap.pkl')

print("Merged DataFrame shape:", merged_df.shape)


In [ ]:
# ----------- Data Preparation -----------

def prepare_prophet_data(df, target_col='Room Revenue', exogenous_cols=None):
    prophet_df = pd.DataFrame({'ds': df.index, 'y': df[target_col]})
    
    # Add exogenous regressors
    if exogenous_cols:
        for col in exogenous_cols:
            if col in df.columns:
                prophet_df[col] = df[col]
    
    # Day of week (one-hot)
    for i in range(7):
        prophet_df[f'day_{i}'] = (prophet_df['ds'].dt.dayofweek == i).astype(int)
    
    # Weekend flag
    prophet_df['is_weekend'] = prophet_df['ds'].dt.dayofweek.isin([5, 6]).astype(int)
    
    # Month and quarter
    prophet_df['month'] = prophet_df['ds'].dt.month
    prophet_df['quarter'] = prophet_df['ds'].dt.quarter
    # prophet_df['day_of_year'] = prophet_df['ds'].dt.dayofyear

    
    # IN holidays
    us_holidays = holidays.IN()
    prophet_df['is_holiday'] = (prophet_df['ds'].dt.date.apply(lambda x: x in us_holidays)).astype(int)
    
    # Rolling stats on target (7 and 14 days)
    if not pd.isna(df[target_col]).all():
        for window in [7,14]:
            prophet_df[f'rolling_mean_{window}d'] = df[target_col].rolling(window=window).mean().values
            prophet_df[f'rolling_std_{window}d'] = df[target_col].rolling(window=window).std().values
    
    
    prophet_df.fillna(method='bfill', inplace=True)
    prophet_df.fillna(method='ffill', inplace=True)

    return prophet_df


